In [27]:
#Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli, pauli_basis, SuperOp, PTM, Operator
from qiskit.providers.aer.noise import NoiseModel, pauli_error
from qiskit.providers.fake_provider import FakeVigoV2
from qiskit.providers.aer import Aer, AerSimulator
from random import random

#python imports
from random import choice, choices
from itertools import product, permutations, cycle
from scipy.optimize import curve_fit, nnls
from matplotlib import pyplot as plt
import numpy as np

plt.style.use("ggplot")

In [28]:
def get_noise_model():
    from random import random, choices
    from qiskit.providers.aer.noise import NoiseModel, pauli_error
    from qiskit.quantum_info import Pauli, pauli_basis
    twoqubit_errorops = [Pauli('IX')]
    twoqubit_errorprobs = [0.05]
    #create normalized error model
    twoqubit_error_template = [(op, p) for op,p in zip(twoqubit_errorops, twoqubit_errorprobs)]+[(Pauli("II"), 1-sum(twoqubit_errorprobs))]
    twoqubit_error = pauli_error(twoqubit_error_template)
    noise_model = NoiseModel()
    #add error model to two-qubit gates.
    noise_model.add_all_qubit_quantum_error(twoqubit_error, ['cx','cz'])
    return (noise_model, twoqubit_error_template, [])

In [33]:
from TrotterExample import executor
import qiskit.providers.fake_provider as fake_provider # FakeMelbourneV2, FakeCasablancaV2, FakeVigoV2, FakeLagosV2, FakeGuadalupeV2, FakeGuadalupe, FakeGeneva

def add_CNOT(prog, num):
    for _ in range(num):
        prog.cnot(0,1)
    return prog

prog = QuantumCircuit(2)
prog = add_CNOT(prog, 16)

prog.measure_all()
backend = Aer.get_backend('qasm_simulator')
(noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
#noise_model = NoiseModel()
counts = executor([prog], backend, 1024, noise_model=noise_model)
print(counts)
print(noise_model)

{'11': 220, '00': 533, '10': 65, '01': 206}
NoiseModel:
  Basis gates: ['cx', 'cz', 'id', 'rz', 'sx']
  Instructions with noise: ['cz', 'cx']
  All-qubits errors: ['cx', 'cz']
